In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate ,MessagesPlaceholder
from langchain_classic.memory import ConversationBufferWindowMemory

In [ ]:
pdf_paths=['./data/rag1.pdf', './data/rag2.pdf' ,'./data/rag3.pdf']
all_docs=[]

for path in pdf_paths:
    loader=PyPDFLoader(path)
    docs =loader.load()
    all_docs.extend(docs)
    
recur_splitter=RecursiveCharacterTextSplitter(chunk_size=1000 ,chunk_overlap=200)
splits=recur_splitter.split_documents(all_docs)

token_splitter=TokenTextSplitter(chunk_size=1000, chunk_overlap=200 ,model_name='gpt-4o-mini')
token_splits=token_splitter.split_documents(all_docs)



model=OpenAIEmbeddings(model='text-embedding-3-small')
recur_embeddings=model.embed_documents([doc.page_content for doc in splits])
token_embeddings=model.embed_documents([doc.page_content for doc in token_splits])

print('Recursive splitter created')
print(f'number of splits: {len(splits)}')
print('Token-based splitter created')
print(f'number of token-based splits: {len(token_splits)}')

Recursive splitter created
number of splits: 181
Token-based splitter created
number of token-based splits: 76


In [3]:
from langchain_community.vectorstores import Chroma,FAISS
import shutil, os

if os.path.exists("./chroma_recur_db"):
    shutil.rmtree("./chroma_recur_db")
    
model=OpenAIEmbeddings(model='text-embedding-3-small')
chroma_db=Chroma.from_documents(
    documents=token_splits,
    embedding=model,
    collection_name='recur_chunks_collection',
    persist_directory='./chroma_recur_db'
)

In [4]:
results=chroma_db.similarity_search('RAG의 장점' ,k=4)

for i,doc in enumerate(results ,1):
    print(f'---- Result {i} ----')
    print(doc.page_content)
    print()

---- Result 1 ----
정천수
102 지식경영연구 제25권 제3호
을 거처 벡터 저장소에 저장된다(Microsoft, 2023). 이러
한 RAG모델 기반 생성형 AI 서비스 구현 흐름은 <그림 
1>과 같다(정천수, 2023d).
2.1.2. RAG기반 Vector Store 유형
RAG시스템을 구축하기 위해서는 지식이 저장되는 벡
터 데이터베이스를 사용하게 되는데, 벡터 데이터베이스
에 대한 일반적인 벡터 파이프라인은 인덱싱(Indexing), 
조회(Querying), 사후처리(Post Processing)와 같은 3단계
를 거친다(Devtorium, 2023). 특히, RAG기반 벡터 저장소
(Vector Store) 저장 유형은 <그림 2>와 같이 모든 소스 
데이터를 사전에 Vector Store에 저장하는 경우와 질문 
시 실시간으로 넣는 경우로 나눌 수 있다(정천수, 2023d).
기업에서는 내부 지식을 Open LLM을 통해 서비스하
게 되었을 때 보안 이슈 때문에 Local LLM을 사용하는 
것이 중요한 이슈이다(정천수, 2023d). 이때 각 구간별로 
가장 잘 처리할 수 있는 Local LLM을 여러 개 구성하여 
사용하는 것이 효과적인데, <그림 2>에서 여러 개의 소
<그림 1> RAG기반 생성형 AI 서비스 구현 흐름
<그림 2> RAG기반 Vector Store 구성 유형 및 처리절차

---- Result 2 ----
정천수
104 지식경영연구 제25권 제3호
LangChain 또는 LlamaIndex 프레임워크는 이러한 전략
을 구현할 수 있도록 각 항목별 라이브러리를 제공하고 
있어 구현을 좀더 쉽게 할 수 있도록 하고 있다
.
2.2.2. Advanced RAG 유형 연구 및 개선 방향
현재 연구되고 있는 대표적인 Advanced RAG 방안을 
살펴보고, 각 유형의 강점과 약점을 분석하여 본 연구의 
방향성을 제시하고자 한다. 
• Self-RAG: 이 방식은 생성된 답변을 다시 검색하여 
관련 정보를 찾고, 

In [5]:
results=chroma_db.similarity_search('기존 RAG방식의 한계' ,k=4)

for i,doc in enumerate(results ,1):
    print(f'---- Result {i} ----')
    print(doc.page_content)
    print()

---- Result 1 ----
정천수
102 지식경영연구 제25권 제3호
을 거처 벡터 저장소에 저장된다(Microsoft, 2023). 이러
한 RAG모델 기반 생성형 AI 서비스 구현 흐름은 <그림 
1>과 같다(정천수, 2023d).
2.1.2. RAG기반 Vector Store 유형
RAG시스템을 구축하기 위해서는 지식이 저장되는 벡
터 데이터베이스를 사용하게 되는데, 벡터 데이터베이스
에 대한 일반적인 벡터 파이프라인은 인덱싱(Indexing), 
조회(Querying), 사후처리(Post Processing)와 같은 3단계
를 거친다(Devtorium, 2023). 특히, RAG기반 벡터 저장소
(Vector Store) 저장 유형은 <그림 2>와 같이 모든 소스 
데이터를 사전에 Vector Store에 저장하는 경우와 질문 
시 실시간으로 넣는 경우로 나눌 수 있다(정천수, 2023d).
기업에서는 내부 지식을 Open LLM을 통해 서비스하
게 되었을 때 보안 이슈 때문에 Local LLM을 사용하는 
것이 중요한 이슈이다(정천수, 2023d). 이때 각 구간별로 
가장 잘 처리할 수 있는 Local LLM을 여러 개 구성하여 
사용하는 것이 효과적인데, <그림 2>에서 여러 개의 소
<그림 1> RAG기반 생성형 AI 서비스 구현 흐름
<그림 2> RAG기반 Vector Store 구성 유형 및 처리절차

---- Result 2 ----
정천수
104 지식경영연구 제25권 제3호
LangChain 또는 LlamaIndex 프레임워크는 이러한 전략
을 구현할 수 있도록 각 항목별 라이브러리를 제공하고 
있어 구현을 좀더 쉽게 할 수 있도록 하고 있다
.
2.2.2. Advanced RAG 유형 연구 및 개선 방향
현재 연구되고 있는 대표적인 Advanced RAG 방안을 
살펴보고, 각 유형의 강점과 약점을 분석하여 본 연구의 
방향성을 제시하고자 한다. 
• Self-RAG: 이 방식은 생성된 답변을 다시 검색하여 
관련 정보를 찾고, 

In [6]:
results=chroma_db.similarity_search('monoT5와 RankLLaMA 중 성능이 더 좋은 것은?' ,k=4)

for i,doc in enumerate(results ,1):
    print(f'---- Result {i} ----')
    print(doc.page_content)
    print()

---- Result 1 ----
0.58, it came
at a computational cost of 11.71 seconds per
query. In practice, the "Hybrid" or "Original"
methods are recommended, as they maintain
comparable performance with reduced latency.
• Reranking Module: Reranking is critical to
maintaining high-quality results, as demonstrated
by a performance drop in its absence. Among
DLM-based rerankers, monoT5 significantly out-
performed monoBERT and RankLLaMA. This
superiority can be attributed to monoT5’s larger
parameter set and more extensive training data, as
well as its encoder-decoder architecture, which
provides enhanced natural language understand-
ing compared to the decoder-only LLaMA model.
MonoT5’s effectiveness in boosting the relevance
of retrieved documents affirms the necessity of
reranking in improving the quality of generated
responses.
• Repacking Module: The Reverse configuration
exhibited superior performance, achieving an
RAG score of 0.560. This highlights the impor-
tance of positioning more re

In [4]:
#실행
model=OpenAIEmbeddings(model='text-embedding-3-small')
chroma_db=Chroma(
    collection_name='recur_chunks_collection',
    persist_directory='./chroma_recur_db',
    embedding_function=model
)
query='monoT5와 RankLLaMA 중 성능이 더 좋은 것은?'
retriever= chroma_db.as_retriever(search_kwargs={'k':2})
docs=retriever.invoke(query)

C:\Users\asguug\AppData\Local\Temp\ipykernel_173552\2680780438.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  chroma_db=Chroma(


In [5]:
#실행
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    '''다음 컨텍스트만 사용해 질문에 답하세요.
    컨텍스트:{context}
    질문:{question}
    '''
)

llm=ChatOpenAI(model_name='gpt-4o-mini' ,temperature=0)
llm_chain=prompt |llm
# result= llm_chain.invoke({'context':docs ,'question':query})

# print(result)

In [ ]:
rewrite_prompt=ChatPromptTemplate.from_template(
    '''
    검색 엔진이 주어진 질문에 답할 수 있도록 더 나은 영문 검색어를 제공하세요. 쿼리는 \'**\'로 끝내세요.
    질문:{x}
    답변:
    '''
)

def parse_rewriter_output(message):
    return message.content.strip('\'').strip('**')
rewriter=rewrite_prompt | llm |parse_rewriter_output

@chain
def qa_rrr(input):
    new_query=rewriter.invoke(input)
    print('재작성한 쿼리:' ,new_query)
    docs=retriever.invoke(new_query)
    formatted =prompt.invoke({'context':docs ,'question':input})
    answer =llm.invoke(formatted)
    return answer

result =qa_rrr.invoke(query)
print(result.content)
    

재작성한 쿼리: "Comparison of performance between monoT5 and RankLLaMA**"
monoT5가 RankLLaMA보다 성능이 더 좋습니다. monoT5는 더 큰 파라미터 집합과 더 광범위한 훈련 데이터를 가지고 있으며, 자연어 이해에서 우수한 성능을 보여줍니다. Reranking 모듈에서 monoT5는 monoBERT와 RankLLaMA보다 뛰어난 성능을 발휘했습니다.


In [6]:
#실행
#다중 쿼리 검색
perspective_prompt= ChatPromptTemplate.from_template(
    '''당신은 AI 언어 모델 어시스턴트입니다.
    주어진 사용자 질문의 다섯 가지 버전을 생성하여 벡터 데이터베이스에서 관련 문서를 검색하세요.
    사용자 질문에 대한 다양한 관점을 생성함으로써 사용자가 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 
    목표입니다.이러한 대체 질문을 개행으로 구분하여 제공하세요.
    원래질문:{question}'''
)
def parse_queries_output(message):
    return message.content.split('\n')

query_gen=perspective_prompt |llm |parse_queries_output

def get_unique_union(document_lists):
    deduped_docs={doc.page_content:doc for sublist in document_lists for doc in sublist}
    return list(deduped_docs.values())

retrieval_chain= query_gen | retriever.batch | get_unique_union

In [ ]:
prompt = ChatPromptTemplate.from_template(
    '''
    당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.
    반드시 제공된 문서 내용을 기반으로 답변하고, 문서에 없는 내용은 ‘문서에 정보가 없습니다’라고 답하세요.
    컨텍스트:{context}
    질문:{question}
    '''
)
query='monoT5와 RankLLaMA 중 성능이 더 좋은 것은?'

@chain
def multi_query_qa(input):
    docs=retrieval_chain.invoke(input)
    formatted=prompt.invoke({'context':docs ,'question':input})
    answer=llm.invoke(formatted)
    return {"answer" :answer ,'docs':docs}

print('다중 쿼리 검색\n')
result= multi_query_qa.invoke(query)
print(f"답변: {result['answer'].content}\n")

print("[근거 문서 정보]")
for i,doc in enumerate(result['docs'] ,1):
    source =doc.metadata.get('source' ,'파일 정보 없음')
    page=doc.metadata.get('page' ,'페이지 정보 없음')
    print(f"{i}. 파일명:{source} ,페이지/섹션:{page}")

다중 쿼리 검색

답변: 문서에 따르면, monoT5는 monoBERT와 RankLLaMA보다 성능이 우수하다고 언급되어 있습니다. 그러나 RankLLaMA는 최고의 성능을 달성하는 데 적합하다고도 설명되어 있습니다. 따라서 두 모델의 성능은 각각의 목적에 따라 다를 수 있습니다.

[근거 문서 정보]
1. 파일명:./data/rag1.pdf ,페이지/섹션:6
2. 파일명:./data/rag1.pdf ,페이지/섹션:7
3. 파일명:./data/rag1.pdf ,페이지/섹션:4
4. 파일명:./data/rag3.pdf ,페이지/섹션:7


In [7]:
#실행
#RAG 융합
def reciprocal_rank_fusion(results:list[list] ,k=60):
    '''여러 순위 문서 목록에 대해 상호 순위 융합 및 RRF 공식에 사용되는 선택적 매개변수 k입니다.
    '''
    fused_scores={}
    documents={}
    for docs in results:
        for rank ,doc in enumerate(docs):
            doc_str=doc.page_content
            if doc_str not in fused_scores:
                fused_scores[doc_str]=0
                documents[doc_str]=doc
            fused_scores[doc_str]+=1/(rank+k)
            
    reranked_doc_strs=sorted(fused_scores ,key=lambda d:fused_scores[d] ,reverse=True)
    return [documents[doc_str] for doc_str in reranked_doc_strs]

retrieval_chain=query_gen | retriever.batch |reciprocal_rank_fusion

In [13]:
#실행
prompt = ChatPromptTemplate.from_template(
    '''
    당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.
    반드시 제공된 문서 내용을 기반으로 답변하고, 문서에 없는 내용은 ‘문서에 정보가 없습니다’라고 답하세요.
    컨텍스트:{context}
    질문:{question}
    '''
)
query='monoT5와 RankLLaMA 중 성능이 더 좋은 것은?'

@chain
def rag_fusion(input):
    docs=retrieval_chain.invoke(input)
    formatted=prompt.invoke({'context':docs ,'question':input})
    answer=llm.invoke(formatted)
    return {"answer" :answer ,'docs':docs}

# print('RAG 융합 실행\n')
# result= rag_fusion.invoke(query)
# print(f"답변: {result['answer'].content}\n")

# print("[근거 문서 정보]")
# for i,doc in enumerate(result['docs'] ,1):
#     source =doc.metadata.get('source' ,'파일 정보 없음')
#     page=doc.metadata.get('page' ,'페이지 정보 없음')
#     print(f"{i}. 파일명:{source} ,페이지/섹션:{page}")

In [14]:
#실행
while True:
    question=input("질문을 입력하세요 (종료:exit) :")
    if question.lower() in ['exit','quit']:
        break
    answer = rag_fusion.invoke(question)
    print(answer['answer'].content)
    print('--------------------------------------')

하이브리드 서치는 두 가지 검색 방식을 사용합니다: 희소 검색(sparse retrieval)과 밀집 검색(dense retrieval)입니다. 희소 검색에는 BM25 알고리즘이 사용되며, 밀집 검색에는 Contriever라는 비지도 대조 텍스트 인코더가 사용됩니다.
--------------------------------------
RapidMiner는 셀프서비스 분석을 지원하는 예측적 데이터 분석 플랫폼으로, 완전 GUI 방식으로 데이터 로딩부터 모델 적용까지 시각적으로 작업 흐름을 설계할 수 있습니다. 비전문가도 쉽게 사용할 수 있도록 설계되어 있어 프로그래밍을 알지 못하더라도 RAG 구축 과정을 직관적으로 이해하고 구출할 수 있습니다.
--------------------------------------
문서에 정보가 없습니다.
--------------------------------------


In [15]:
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_core.output_parsers import StrOutputParser

memory=ConversationBufferWindowMemory(k=3 ,return_messages=True ,memory_key='chat_history')

condense_prompt = ChatPromptTemplate.from_template(
    """이전 대화 내역과 최신 사용자 질문이 주어졌을 때, 
    이 질문이 이전 대화 맥락을 필요로 한다면 질문과 이전 대화 목록을 같이 넘겨주세요. 
    대화 내역: {chat_history}
    질문: {question}
    독립적인 질문:"""
)
condense_chain = condense_prompt | llm | StrOutputParser()

prompt=ChatPromptTemplate.from_messages([
    ("system" ,'''
     당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.
    반드시 제공된 문서 내용을 기반으로 답변하고, 문서에 없는 내용은 ‘문서에 정보가 없습니다’라고 답하세요.
    이전 대화 기록을 참고하여 자연스럽게 답변하세요.
    '''),
    MessagesPlaceholder(variable_name='chat_history'),
    ("human" ,"컨텍스트:{context}\n질문:{question}")
])

@chain
def rag_fusion_with_memory(input):
    history=memory.load_memory_variables({})['chat_history']
    
    if history:
        search_query=condense_chain.invoke({"chat_history":history ,"question":input})
    else:
        search_query=input
    print(f"검색용 커리: {search_query}")
    
    docs=retrieval_chain.invoke(search_query)
    
    formatted=prompt.invoke({
        'context':docs,
        'question':input,
        'chat_history':history
    })
    answer=llm.invoke(formatted)
    memory.save_context(
        {'question':input},
        {'output':answer.content}
    )
    return {"answer":answer ,'docs':docs}

print('RAG 융합 (메모리 적용) 실행\n')

RAG 융합 (메모리 적용) 실행



In [16]:
#실행
while True:
    question = input("질문을 입력하세요 (종료: exit): ")
    if question.lower() in ["exit", "quit"]:
        break
    answer = rag_fusion_with_memory.invoke(question)
    print(answer['answer'].content)

검색용 커리: 'Hybrid Search'는 어떤 두 가지 검색 방식을 결합한 것인가요? 
'Hybrid Search'는 sparse retrieval 방식인 BM25와 dense retrieval 방식인 Original embedding을 결합한 것입니다.
검색용 커리: 이 질문은 이전 대화 맥락을 필요로 합니다. 따라서 질문과 이전 대화 목록을 함께 제공합니다.

대화 내역:
1. Human: "'Hybrid Search'는 어떤 두 가지 검색 방식을 결합한 것인가요?"
2. AI: "'Hybrid Search'는 sparse retrieval 방식인 BM25와 dense retrieval 방식인 Original embedding을 결합한 것입니다."

질문: 이 방식과 HyDE를 함께 사용했을 때의 장단점은 무엇인가요?
Hybrid Search와 HyDE를 함께 사용했을 때의 장점은 성능 향상입니다. HyDE는 가상의 문서를 생성하여 검색 성능을 크게 향상시킬 수 있으며, Hybrid Search는 BM25와 Original embedding을 결합하여 효율적인 검색을 제공합니다. 이 조합은 높은 성능을 유지하면서도 상대적으로 낮은 지연 시간을 달성할 수 있습니다.

단점으로는, 가상의 문서를 여러 개 연결하는 경우 검색 성능이 향상될 수 있지만, 지연 시간이 증가하는 트레이드오프가 존재합니다. 또한, 가상의 문서 수를 무작정 늘리는 것은 큰 이점을 주지 않으며, 오히려 지연 시간을 상당히 증가시킬 수 있습니다. 따라서, HyDE와 Hybrid Search를 사용할 때는 적절한 가상의 문서 수를 선택하는 것이 중요합니다.
